# Mercator - Isopycnais

#### Packages

In [1]:
import xarray as xr
import numpy  as np

import glob
import os

import warnings
warnings.filterwarnings("ignore")

#### Importing Dataset

In [2]:
filenames = glob.glob("d:/00_Masters/01_Dados/Mercator/*.nc")
ds = xr.open_mfdataset(filenames)
_, index = np.unique(ds['time'], return_index=True)
ds = ds.isel(time=index)

#### Calculating Isoterm of 23.3°C Depths

    Profundidade mínima da ACAS 
        Limite ACAS-TEMP

In [3]:
depth = ds['depth'].to_dataframe()
depth.reset_index(drop=True,inplace=True,)
ds['thetao'] = ds['thetao'].fillna(-999999) # mask nan / Land

#### Min. Depth

In [19]:
TEMP = ds.map(lambda p: ds.thetao <= 23.3) # Return array with boolean % similiar to where
temp = TEMP.thetao.values

isoterm = np.zeros((TEMP.time.size,TEMP.latitude.size,TEMP.longitude.size)) 
 # loop through time indices
for ti in np.arange(0,TEMP.time.size,1):
    # loop through lat,lon indices
    for yi in np.arange(0,TEMP.latitude.size,1):        
        for xi in np.arange(0,TEMP.longitude.size,1):
            # Condition for TEMP
            iso_depth = []
            for i in np.arange(29):
                if (i < 28) and ((temp[ti][i][yi][xi] == True) and (temp[ti][i+1][yi][xi] == False)):
                    iso_depth = depth.loc[i]
                    break
                else:
                    iso_depth = np.nan          
            

            isoterm[ti,yi,xi] = iso_depth

In [30]:
ds['IsoTerm_23_depth'] = (['time','latitude','longitude'],isoterm)
ds['IsoTerm_23_depth'].attrs["units"] = "m"
ds['IsoTerm_23_depth'].attrs["unit_long"] = "meters"
ds['IsoTerm_23_depth'].attrs["long_name"] = "Isoterm Depth"
ds['IsoTerm_23_depth'].attrs["standard_name"] = "Isoterm_depth"

#### Exporting TW_Depth as NC

In [33]:
ds['IsoTerm_23_depth'].to_netcdf('D:/00_Masters/01_Dados/IsoTerm_23_depth.nc')

-----

#### Max. Depth

In [5]:
TEMP = ds.map(lambda p: ds.thetao > 23.3) # Return array with boolean % similiar to where
temp = TEMP.thetao.values

isoterm = np.zeros((TEMP.time.size,TEMP.latitude.size,TEMP.longitude.size)) 
 # loop through time indices
for ti in np.arange(0,TEMP.time.size,1):
    # loop through lat,lon indices
    for yi in np.arange(0,TEMP.latitude.size,1):        
        for xi in np.arange(0,TEMP.longitude.size,1):
            # Condition for TEMP
            iso_depth = []
            for i in np.arange(1,29):
                if (i < 28) and ((temp[ti][i-1][yi][xi] == False) and (temp[ti][i][yi][xi] == True)):
                    iso_depth = depth.loc[i]
                    break
                else:
                    iso_depth = np.nan          
            

            isoterm[ti,yi,xi] = iso_depth

In [6]:
ds['IsoTerm_23mais_depth'] = (['time','latitude','longitude'],isoterm)
ds['IsoTerm_23mais_depth'].attrs["units"] = "m"
ds['IsoTerm_23mais_depth'].attrs["unit_long"] = "meters"
ds['IsoTerm_23mais_depth'].attrs["long_name"] = "Isoterm Depth"
ds['IsoTerm_23mais_depth'].attrs["standard_name"] = "Isoterm_depth"

In [8]:
ds['IsoTerm_23mais_depth'].to_netcdf('D:/00_Masters/01_Dados/IsoTerm_23mais_depth.nc')

-----